In [1]:
#import libraries
import os
import pandas as pd
import re
import csv
from io import StringIO
from pathlib import Path

In [2]:
#read file
filename = Path(input("Enter the file name : "))

Enter the file name : /Users/gio/Google Drive/Money/Briony/HSBC Advance Statements 2019-12-13 to 2020-07-15 - Raw.csv


In [3]:
#load csv to dataframe
col_list = ['date_raw', 'desc_raw', 'credit_raw', 'debit_raw', 'bal_raw']
df = pd.read_csv(filename, names = col_list)

In [4]:
#clean up date field
new = df["date_raw"].str.split(" ", n = 1, expand = True)

# making separate first name column from new data frame 
df["date"]= new[0] 
  
# making separate last name column from new data frame 
df["desc01"]= new[1] 
  
# Dropping old Name columns 
#df.drop(columns =["Name"], inplace = True)

In [5]:
#fill blank dates
df.date = df.date.fillna(method = 'ffill')

In [6]:
#drop last 3 rows
df.drop(df.tail(3).index,inplace=True)

In [7]:
#fix date
#extract day month year
day = df.date.str.extract(r'(\d{2})', expand = False)
month = df.date.str.extract(r'([A-Za-z]{3})', expand = False)
year = df.date.str.extract(r'(\d{4})', expand = False)
# refill column
df['date'] = pd.to_datetime((month + ' ' + day + ", " + year)).dt.strftime("%m/%d/%Y")


In [8]:
#fix desc
#fix nan
df[['desc01','desc_raw']] = df[['desc01','desc_raw']].fillna('') 
#fix \n
df['desc_raw'] = df.desc_raw.str.replace('\n',' ')
df['desc'] = (df.desc01 + ' ' + df.desc_raw).str.lower()

In [9]:
#drop unneeded rows
df = df.drop(df[(df.debit_raw.isna() & df.credit_raw.isna())].index)

In [10]:
#get rid of NaNs
df = df.fillna(value = 0)

In [11]:
#drop unneeded columns
df.drop(['date_raw','desc_raw','bal_raw','desc01'], axis = 1, inplace = True)

In [12]:
#rearrage
df = df[['date','desc','debit_raw','credit_raw']]

In [13]:
#drop unneeded rows
df = df.loc[-df.debit_raw.str.contains('\n',na=False)]

In [14]:
#strip unneeded spaces
df.desc = df.desc.str.strip()
df.desc = df.desc.str.replace(" +"," ")

In [51]:
#Get Currency Rate USD PHP
import re
import requests

CODES = {"USD": "United States Dollar", "PHP": "Philippine Peso", "AUD": "Australian Dollar"}
currency_to_name   = CODES[currency_to]

url = "http://216.58.221.46/search?q=convert+{amount}+{frm}+to+{to}&hl=en&lr=lang_en".format(amount = str('1'),
                                                                                                 frm    = "USD",
                                                                                                 to     = "PHP")
response = requests.get(url, headers={"Range": "bytes=0-1"})
html = response.text
results = re.findall("[\d*\,]*\.\d* {currency_to_name}".format(currency_to_name=currency_to_name), html)
usd_php = float(re.findall("[\d*\,]*\.\d*",results[0])[0])

In [15]:
df

,date,desc,debit_raw,credit_raw
2,12/17/2019,lp mka9004v8 hib- 863771153x3dswzi9ph christma...,"5,012.41",0
3,12/26/2019,tax from ssv ref zdd4-90012,0.3,0
4,12/26/2019,credit interest ref zdd4-90012,0,1.98
5,01/06/2020,pay by 061-032215-130 to 4363 6700 1003 2434 r...,673.01,0
9,01/06/2020,ref yiro-00434,0,786.6
14,01/22/2020,lp mka000139 hib- 914518746x4hlousaql citibank...,"5,880.95",0
15,01/22/2020,ocmt/usd5990.25 asian development bank ref: ta...,0,"5,978.25"
21,01/29/2020,briony rae eales 2596232913 ayala triangle ref...,"5,797.44",0
23,02/03/2020,ref gs08-07074 pay by 061-032215-130 to 4363 6...,"5,768.80",0
24,02/03/2020,ref zcpa-00031 ocmt/usd7701.75 asian developme...,999.01,0


In [ ]:
#export to csv
filename_csv_full = 'for import ' + filename.name
export_csv = df.to_csv (filename_csv_full, index = None, header=True)